In [1]:
import os
import glob
from torch.utils import data
import torch
import torch.nn as nn
from torch import optim


from ncg.image_caption_dataset import ImageCaptionDataset
from ncg.models.train_model import train_iter
from ncg.models.show_and_tell import ShowAndTell

In [29]:
fpattern_image_encodings = os.path.join('output', 'demo', 'resnet18_avgpool', '*.jpg.pt')
fpaths_image_encodings = glob.glob(fpattern_image_encodings)
# TODO image splits
# TODO val images not in demo data set

fpattern_captions = os.path.join('output', 'demo', "train.[0-9].en.pt")
fpaths_captions = glob.glob(fpattern_captions)
fpaths_captions = [os.path.join('output', 'demo', "train.1.en.pt")]

In [30]:
dataset_train = ImageCaptionDataset(
    fpaths_image_encodings, 
    fpaths_captions)
dataloader_train = data.DataLoader(dataset_train, batch_size = 1)

In [31]:
vocab = torch.load(os.path.join('output', 'demo', 'vocab.pt'))
vocab_size = vocab.vocab.n_words

im1 = torch.load(fpaths_image_encodings[0])
encoding_size = im1.size()[0]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_criterion = nn.NLLLoss()
learning_rate = 0.005
max_epochs = 5

# TODO: train iter everything moves to GPU
decoder = ShowAndTell(encoding_size, vocab_size, device)
optimizer = optim.SGD(decoder.decoder.parameters(), lr = learning_rate)

In [32]:
token_losses = []
def fn_on_update(epoch, i, token_loss):
    token_losses.append(token_loss)
    if i % 100 == 0:
        new_losses = token_losses[-100:]
        print(epoch, i, sum(new_losses)/len(new_losses))

In [33]:
batch_losses = train_iter(decoder, dataloader_train, loss_criterion, 
                          optimizer, device, max_epochs, fn_on_update)

1 100 7.181354255676269
1 200 6.1093105030059816
1 300 5.688761115074158
1 400 5.191423473358154
1 500 5.230095751285553
1 600 5.0970507645606995
1 700 4.940425279140473
1 800 4.79053778886795
1 900 4.885478141307831
1 1000 4.825052509307861
2 100 4.700515766143798
2 200 4.711035912036896
2 300 4.733883557319641
2 400 4.436252694129944
2 500 4.668575603961944
2 600 4.659923768043518
2 700 4.529715754985809
2 800 4.422660655975342
2 900 4.571345474720001
2 1000 4.550533931255341
3 100 4.436657981872559
3 200 4.467076804637909
3 300 4.505985186100006
3 400 4.199919283390045
3 500 4.474245839118957
3 600 4.473096518516541
3 700 4.346262171268463
3 800 4.24333487033844
3 900 4.405389385223389
3 1000 4.3986550116539
4 100 4.283857326507569
4 200 4.319849865436554
4 300 4.361913802623749
4 400 4.051162161827087
4 500 4.341057794094086
4 600 4.338972442150116
4 700 4.215483071804047
4 800 4.116042544841767
4 900 4.278644165992737
4 1000 4.283731174468994
5 100 4.163591239452362
5 200 4.205843

In [ ]:
len(dataloader_train)

In [11]:
len(token_losses)


4067

In [15]:
token_losses[4::1000]

[6.587995529174805,
 2.8842263221740723,
 3.233182191848755,
 3.5987658500671387,
 4.11681604385376]

In [27]:
encoding_size

512